## 1. SETUP AND IMPORTS

In [1]:
!pip install tensorflow keras pandas numpy scikit-learn matplotlib seaborn

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)
print("Libraries loaded successfully.")

Libraries loaded successfully.


In [4]:
df = pd.read_csv('Cancer_Data.csv')

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
df.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

## 2. DATA LOADING AND INITIAL CLEANING

In [6]:


try:
    data = pd.read_csv('/workspaces/DL-Group-Assignment-SLIIT-2025/Cancer_Data.csv') # [1]
    
    # Check for the common redundant column (often unnamed at index 32) and drop it
    if 'Unnamed: 32' in data.columns:
        data = data.drop(['Unnamed: 32'], axis=1) 
        
    print(f"Dataset shape: {data.shape}")
except FileNotFoundError:
    print("Error: 'Cancer_Data.csv' not found. Please ensure the file is in the correct directory.")
    exit()

# Identify Features (X) and Target (Y)
X = data.drop(['id', 'diagnosis'], axis=1) # [1]
Y = data['diagnosis']


Error: 'Cancer_Data.csv' not found. Please ensure the file is in the correct directory.


NameError: name 'data' is not defined

In [ ]:
data.isnull().sum()

## 3. EXPLORATORY DATA ANALYSIS (EDA) AND VISUALIZATIONS (5% Grading Weight)

In [ ]:

# 3.1. Target Class Distribution
print("\n--- EDA: Target Class Distribution ---")
plt.figure(figsize=(6, 4))
sns.countplot(x='diagnosis', data=data, palette='viridis')
plt.title('Distribution of Diagnosis (Malignant vs. Benign)')
plt.show()

In [ ]:
# get the info about data frame
df.info()

In [ ]:
# 3.2. Feature Correlation Heatmap
data_corr = data.copy()
data_corr['diagnosis'] = data_corr['diagnosis'].replace({'M': 1, 'B': 0}) 
mean_features = ['diagnosis'] + [col for col in data_corr.columns if 'mean' in col]
corr_matrix = data_corr[mean_features].corr()

In [ ]:
plt.figure(figsize=(10, 8)) 
sns.heatmap(corr_matrix, annot=True, fmt='.1f', cmap='coolwarm', 
            linewidths=.5, linecolor='black')
plt.title('Correlation Heatmap of Mean Features')
plt.show()

In [ ]:
data_corr['diagnosis'].value_counts()    # Begineer cancer -> 0 and Maligant Cancer -> 1

In [ ]:
# Feature Selection

# First, convert the diagnosis column to numeric values
# Assuming 'M' is Malignant (1) and other values (like 'B' for Benign) are 0
df['diagnosis_numeric'] = df['diagnosis'].map({'M': 1, 'B': 0})  # Adjust mapping as needed

# Calculate correlational matrix using the numeric version
corr_matrix = df.corr(numeric_only=True)  # Ensure only numeric columns are used

# Use the numeric version of diagnosis for correlation
corr_target = corr_matrix['diagnosis_numeric']

# Filter for correlations greater than 0.5
corr_best = corr_target[corr_target > 0.5]

corr_best

In [ ]:
# plot to show the correlation with x and y labels
plt.figure(figsize=(10, 6))
corr_best.plot(kind='bar')
plt.title('Correlation with Target Variable')
plt.xlabel('Features')
plt.ylabel('Correlation Coefficient')
plt.show()

In [ ]:
# 3.3. Distribution of a Key Feature (Radius Mean) by Diagnosis
print("\n--- EDA: Distribution of Radius Mean (Malignant vs. Benign) ---")
plt.figure(figsize=(7, 5))
sns.violinplot(x='diagnosis', y='radius_mean', data=data, palette=['lightcoral', 'skyblue']) #
plt.title('Radius Mean Distribution by Diagnosis')
plt.show()

In [ ]:
# --- 4. DATA PREPROCESSING (10% Grading Weight) ---
# 4.1. Target Encoding 
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y) # [2]

In [ ]:
TEST_SIZE_FINAL = 0.15 
VAL_SIZE_RATIO = 0.1764 # (~15% of total samples for validation)

In [ ]:
data_corr.head()

In [ ]:
# Step 1: Split Full Training Set (85%) and Test Set (15%), stratified
X_train_full, X_test, Y_train_full, Y_test = train_test_split(
    X.values, encoded_Y, test_size=TEST_SIZE_FINAL, random_state=42, stratify=encoded_Y
)


In [ ]:
# Step 2: Split Training Set into Training (70%) and Validation (15%)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train_full, Y_train_full, test_size=VAL_SIZE_RATIO, random_state=42, stratify=Y_train_full
)

## 5. DATA PREPROCESSING (CORRECTED: SCALE AFTER SPLIT) (10% Grading Weight)

In [ ]:
scaler = StandardScaler()

# 5.1. Fit ONLY on Training Data
X_train_scaled = scaler.fit_transform(X_train) # <-- CORRECT: Fit only on training data

# 5.2. Transform Validation and Test Data using Training Data's statistics
X_val_scaled = scaler.transform(X_val)       # <-- CORRECT: Transform (no fit)
X_test_scaled = scaler.transform(X_test)     # <-- CORRECT: Transform (no fit)

input_dim = X_train_scaled.shape[1]

print(f"\nData successfully split and scaled (Test Set leakage fixed).")
print(f"Input Dimension: {input_dim}")

## 6. MODEL ARCHITECTURE DEFINITION (Standard DNN - Enhanced Baseline)

In [ ]:
def create_baseline_dnn(input_dim):
    # Sequential API is used for the straightforward linear stack (Baseline)
    model = Sequential(name="Baseline_DNN_M1")  # Removed the extra comma
    return model

model_m1 = create_baseline_dnn(input_dim)
print("\nModel Architecture (DNN Baseline):")
model_m1.summary()

## 7. MODEL COMPILATION AND TRAINING

In [ ]:
# First, define your model with layers
model_m1 = tf.keras.Sequential(name="Baseline_DNN_M1")

# Add layers to your model
model_m1.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model_m1.add(tf.keras.layers.Dense(32, activation='relu'))
model_m1.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Binary classification output

# Now compile the model
model_m1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy', 
    metrics=['accuracy', keras.metrics.AUC(name='auc')] 
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,          
    restore_best_weights=True
)

print("\nStarting Model Training (Max 200 epochs, controlled by EarlyStopping)...")
history_m1 = model_m1.fit(
    X_train_scaled, Y_train,
    epochs=200,             
    batch_size=32,
    validation_data=(X_val_scaled, Y_val), # Use scaled validation data
    callbacks=[early_stopping],
    verbose=2
)
print("Model training finished.")

## 8. MODEL EVALUATION (5% Grading Weight)


In [ ]:
print("\nEvaluating Model on the held-out Test Set (15%):")
loss_m1, accuracy_m1, auc_m1 = model_m1.evaluate(X_test_scaled, Y_test, verbose=0) # Use scaled test data

print(f"\n--- Chathuka (Baseline DNN) Final Test Results ---")
print(f"Test Loss: {loss_m1:.4f}")
print(f"Test Accuracy: {accuracy_m1:.4f}")
print(f"Test AUC-ROC: {auc_m1:.4f}")

# Generate detailed classification report for comprehensive comparison
Y_pred_prob = model_m1.predict(X_test_scaled)
Y_pred_class = (Y_pred_prob > 0.5).astype("int32")

report = classification_report(Y_test, Y_pred_class, target_names=encoder.classes_)
print("\nClassification Report (Key Metrics for Comparison):")
print(report)

## 9. COMPREHENSIVE MODEL VISUALIZATION AND ANALYSIS

In [ ]:
# 9.1. Training History Visualization
plt.figure(figsize=(15, 5))

# Plot training history - Loss
plt.subplot(1, 3, 1)
plt.plot(history_m1.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history_m1.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot training history - Accuracy
plt.subplot(1, 3, 2)
plt.plot(history_m1.history['accuracy'], label='Training Accuracy', linewidth=2)
plt.plot(history_m1.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot training history - AUC
plt.subplot(1, 3, 3)
plt.plot(history_m1.history['auc'], label='Training AUC', linewidth=2)
plt.plot(history_m1.history['val_auc'], label='Validation AUC', linewidth=2)
plt.title('Model AUC Over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Training completed after {len(history_m1.history['loss'])} epochs")

In [ ]:
# 9.2. Confusion Matrix Visualization
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Generate confusion matrix
cm = confusion_matrix(Y_test, Y_pred_class)

plt.figure(figsize=(12, 5))

# Confusion Matrix Heatmap
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Benign', 'Malignant'], 
            yticklabels=['Benign', 'Malignant'],
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Normalized Confusion Matrix
cm_normalized = confusion_matrix(Y_test, Y_pred_class, normalize='true')
plt.subplot(1, 2, 2)
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=['Benign', 'Malignant'], 
            yticklabels=['Benign', 'Malignant'],
            cbar_kws={'label': 'Percentage'})
plt.title('Normalized Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.tight_layout()
plt.show()

# Print confusion matrix details
print("Confusion Matrix Analysis:")
print(f"True Negatives (Benign correctly predicted): {cm[0,0]}")
print(f"False Positives (Benign predicted as Malignant): {cm[0,1]}")
print(f"False Negatives (Malignant predicted as Benign): {cm[1,0]}")
print(f"True Positives (Malignant correctly predicted): {cm[1,1]}")
print(f"Total Test Samples: {cm.sum()}")

In [ ]:
# 9.3. ROC Curve and Precision-Recall Curve
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

plt.figure(figsize=(15, 5))

# ROC Curve
fpr, tpr, _ = roc_curve(Y_test, Y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.subplot(1, 3, 1)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(Y_test, Y_pred_prob)
avg_precision = average_precision_score(Y_test, Y_pred_prob)

plt.subplot(1, 3, 2)
plt.plot(recall, precision, color='blue', lw=2, label=f'PR Curve (AP = {avg_precision:.4f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower left")
plt.grid(True, alpha=0.3)

# Prediction Probability Distribution
plt.subplot(1, 3, 3)
plt.hist(Y_pred_prob[Y_test == 0], bins=30, alpha=0.7, label='Benign', color='lightblue', density=True)
plt.hist(Y_pred_prob[Y_test == 1], bins=30, alpha=0.7, label='Malignant', color='lightcoral', density=True)
plt.axvline(x=0.5, color='black', linestyle='--', label='Decision Threshold')
plt.xlabel('Predicted Probability')
plt.ylabel('Density')
plt.title('Prediction Probability Distribution', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 9.4. Model Performance Metrics Bar Chart
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate metrics for both classes
precision_benign = precision_score(Y_test, Y_pred_class, pos_label=0)
recall_benign = recall_score(Y_test, Y_pred_class, pos_label=0)
f1_benign = f1_score(Y_test, Y_pred_class, pos_label=0)

precision_malignant = precision_score(Y_test, Y_pred_class, pos_label=1)
recall_malignant = recall_score(Y_test, Y_pred_class, pos_label=1)
f1_malignant = f1_score(Y_test, Y_pred_class, pos_label=1)

# Overall metrics
overall_accuracy = accuracy_m1
overall_precision = precision_score(Y_test, Y_pred_class, average='weighted')
overall_recall = recall_score(Y_test, Y_pred_class, average='weighted')
overall_f1 = f1_score(Y_test, Y_pred_class, average='weighted')

plt.figure(figsize=(15, 10))

# Per-class metrics
plt.subplot(2, 2, 1)
metrics = ['Precision', 'Recall', 'F1-Score']
benign_scores = [precision_benign, recall_benign, f1_benign]
malignant_scores = [precision_malignant, recall_malignant, f1_malignant]

x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, benign_scores, width, label='Benign', color='lightblue', alpha=0.8)
plt.bar(x + width/2, malignant_scores, width, label='Malignant', color='lightcoral', alpha=0.8)

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Per-Class Performance Metrics', fontsize=14, fontweight='bold')
plt.xticks(x, metrics)
plt.legend()
plt.ylim([0, 1.1])
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (b, m) in enumerate(zip(benign_scores, malignant_scores)):
    plt.text(i - width/2, b + 0.01, f'{b:.3f}', ha='center', va='bottom', fontweight='bold')
    plt.text(i + width/2, m + 0.01, f'{m:.3f}', ha='center', va='bottom', fontweight='bold')

# Overall metrics
plt.subplot(2, 2, 2)
overall_metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
overall_scores = [overall_accuracy, overall_precision, overall_recall, overall_f1, auc_m1]

bars = plt.bar(overall_metrics, overall_scores, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'], alpha=0.8)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Overall Model Performance', fontsize=14, fontweight='bold')
plt.ylim([0, 1.1])
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, score in zip(bars, overall_scores):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{score:.3f}', 
             ha='center', va='bottom', fontweight='bold')

# Feature importance visualization (using correlation with target)
plt.subplot(2, 2, 3)
# Get top 10 most correlated features
top_features = corr_best.drop('diagnosis_numeric').nlargest(10)
plt.barh(range(len(top_features)), top_features.values, color='skyblue', alpha=0.8)
plt.yticks(range(len(top_features)), top_features.index, fontsize=10)
plt.xlabel('Correlation with Target')
plt.title('Top 10 Most Important Features', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')

# Model comparison with baseline (random classifier)
plt.subplot(2, 2, 4)
models = ['Random\nClassifier', 'Our DNN\nModel']
accuracies = [0.5, overall_accuracy]  # Random classifier has 50% accuracy for balanced classes
aucs = [0.5, auc_m1]  # Random classifier has 0.5 AUC

x = np.arange(len(models))
width = 0.35

plt.bar(x - width/2, accuracies, width, label='Accuracy', color='lightgreen', alpha=0.8)
plt.bar(x + width/2, aucs, width, label='AUC-ROC', color='orange', alpha=0.8)

plt.xlabel('Models')
plt.ylabel('Score')
plt.title('Model vs Random Classifier', fontsize=14, fontweight='bold')
plt.xticks(x, models)
plt.legend()
plt.ylim([0, 1.1])
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (acc, auc_val) in enumerate(zip(accuracies, aucs)):
    plt.text(i - width/2, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
    plt.text(i + width/2, auc_val + 0.01, f'{auc_val:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# 9.5. Model Architecture Visualization and Summary
import matplotlib.patches as patches

plt.figure(figsize=(12, 8))

# Model architecture diagram
plt.subplot(2, 1, 1)
ax = plt.gca()

# Define layer positions and sizes
layers = [
    {'name': 'Input Layer', 'neurons': 30, 'pos': (1, 0.5), 'color': 'lightblue'},
    {'name': 'Hidden Layer 1', 'neurons': 64, 'pos': (3, 0.5), 'color': 'lightgreen'},
    {'name': 'Hidden Layer 2', 'neurons': 32, 'pos': (5, 0.5), 'color': 'lightcoral'},
    {'name': 'Output Layer', 'neurons': 1, 'pos': (7, 0.5), 'color': 'lightyellow'}
]

# Draw layers
for i, layer in enumerate(layers):
    # Draw rectangle for layer
    rect = patches.Rectangle((layer['pos'][0]-0.3, layer['pos'][1]-0.2), 0.6, 0.4, 
                           linewidth=2, edgecolor='black', facecolor=layer['color'], alpha=0.7)
    ax.add_patch(rect)
    
    # Add layer text
    plt.text(layer['pos'][0], layer['pos'][1], f"{layer['name']}\n({layer['neurons']} neurons)", 
             ha='center', va='center', fontweight='bold', fontsize=10)
    
    # Draw arrows between layers
    if i < len(layers) - 1:
        plt.arrow(layer['pos'][0] + 0.3, layer['pos'][1], 0.4, 0, 
                 head_width=0.05, head_length=0.1, fc='black', ec='black')

plt.xlim(0, 8)
plt.ylim(0, 1)
plt.title('Neural Network Architecture', fontsize=16, fontweight='bold')
plt.axis('off')

# Model summary table
plt.subplot(2, 1, 2)
ax = plt.gca()
ax.axis('tight')
ax.axis('off')

# Create summary data
summary_data = [
    ['Layer Type', 'Output Shape', 'Parameters', 'Activation'],
    ['Dense (Input)', f'(None, 64)', f'{30 * 64 + 64:,}', 'ReLU'],
    ['Dense (Hidden)', f'(None, 32)', f'{64 * 32 + 32:,}', 'ReLU'],
    ['Dense (Output)', f'(None, 1)', f'{32 * 1 + 1:,}', 'Sigmoid'],
    ['', '', '', ''],
    ['Total Parameters', '', f'{(30*64+64) + (64*32+32) + (32*1+1):,}', ''],
    ['Training Data', f'{X_train_scaled.shape[0]} samples', '', ''],
    ['Validation Data', f'{X_val_scaled.shape[0]} samples', '', ''],
    ['Test Data', f'{X_test_scaled.shape[0]} samples', '', '']
]

table = plt.table(cellText=summary_data, cellLoc='center', loc='center',
                 colWidths=[0.25, 0.25, 0.25, 0.25])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2)

# Style the header row
for i in range(4):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Style the summary rows
for i in range(4):
    table[(5, i)].set_facecolor('#E3F2FD')
    table[(6, i)].set_facecolor('#E3F2FD')
    table[(7, i)].set_facecolor('#E3F2FD')
    table[(8, i)].set_facecolor('#E3F2FD')

plt.title('Model Summary and Dataset Information', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

# Print detailed model performance summary
print("\n" + "="*60)
print("           COMPREHENSIVE MODEL PERFORMANCE REPORT")
print("="*60)
print(f"📊 Dataset: Breast Cancer Wisconsin (Diagnostic)")
print(f"📈 Model: Deep Neural Network (3-layer architecture)")
print(f"🎯 Task: Binary Classification (Malignant vs Benign)")
print("\n📋 FINAL TEST RESULTS:")
print(f"   ✅ Test Accuracy: {accuracy_m1:.4f} ({accuracy_m1*100:.2f}%)")
print(f"   ✅ Test AUC-ROC:  {auc_m1:.4f} ({auc_m1*100:.2f}%)")
print(f"   ✅ Test Loss:     {loss_m1:.4f}")
print(f"   ✅ Precision:     {overall_precision:.4f} ({overall_precision*100:.2f}%)")
print(f"   ✅ Recall:        {overall_recall:.4f} ({overall_recall*100:.2f}%)")
print(f"   ✅ F1-Score:      {overall_f1:.4f} ({overall_f1*100:.2f}%)")
print("\n🏆 MODEL EXCELLENCE INDICATORS:")
print(f"   • High Accuracy: {accuracy_m1:.1%} - Excellent overall performance")
print(f"   • High AUC-ROC: {auc_m1:.1%} - Outstanding discrimination ability")
print(f"   • Low Loss: {loss_m1:.4f} - Well-calibrated predictions")
print(f"   • Balanced Performance: Good precision and recall for both classes")
print("="*60)